In [10]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from selenium import webdriver
import time


## scrape products

In [11]:
# Initialize a headless browser
driver = webdriver.Chrome()
driver.get("https://palmaryfood.com/?page_id=345&lang=en")


time.sleep(3)
for _ in range(3):

    # Scroll down the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait for new content to load
    time.sleep(10)
    
# Get the page source 
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

#get all products info
product_divs = soup.find_all("a",class_="cz_grid_link")

products = []

for product_div in product_divs:

    product_image = product_div.find("img").get("src") #get the product's image link 
    product_name = product_div.find("div",class_="cz_grid_details").find("h3").text.lower() #get the product name

    products.append([product_name,product_image])

# driver.quit()

In [12]:
print(products)
len(products)

[['dondy – almond', 'https://palmaryfood.com/wp-content/uploads/2021/05/Dondy-amande-600x600.jpg'], ['dondy – caramel', 'https://palmaryfood.com/wp-content/uploads/2021/05/Dondy-choco-600x600.jpg'], ['dondy – coconut', 'https://palmaryfood.com/wp-content/uploads/2021/05/Dondy-coco-600x600.jpg'], ['dondy – raspberry', 'https://palmaryfood.com/wp-content/uploads/2021/05/Dondy-framboise-600x600.jpg'], ['dondy – strawberry', 'https://palmaryfood.com/wp-content/uploads/2021/05/Dondy-fraise-600x600.jpg'], ['dream – choco', 'https://palmaryfood.com/wp-content/uploads/2021/05/dream-choco-600x600.jpg'], ['dream – raspberry', 'https://palmaryfood.com/wp-content/uploads/2021/05/dream-framboise-600x600.jpg'], ['dream – strawberry', 'https://palmaryfood.com/wp-content/uploads/2021/05/dream-fraise-600x600.jpg'], ['dream – vanilia', 'https://palmaryfood.com/wp-content/uploads/2021/05/dream-vanille-600x600.jpg'], ['kremali', 'https://palmaryfood.com/wp-content/uploads/2021/05/Kremali-blanc-choco-600x6

102

In [22]:
product_dict = dict()

product_dict["regalo"]=[p for p in products if p[0].startswith("regalo")]

product_dict["kool"]=[p for p in products if p[0].startswith("kool")]

prefixes = ["dondy","kremali","dream","mega","mon","sandwich"]
product_dict["palmary"]= [p for p in products if any(p[0].startswith(prefixe) for prefixe in prefixes)]

prefixes = ["bar","chocolate","coated","cook","filled","wafer","tablet","maxi","mini"]
product_dict["maxon"]= [p for p in products if any(p[0].startswith(prefixe) for prefixe in prefixes)]

product_dict["moment"]=[p for p in products if p[0].startswith("moment")]


In [23]:
product_dict

{'regalo': [['regalo butter',
   'https://palmaryfood.com/wp-content/uploads/2021/04/Regalo-beurre-600x600.jpg'],
  ['regalo cookies',
   'https://palmaryfood.com/wp-content/uploads/2021/05/Cooki-600x600.jpg'],
  ['regalo digestive',
   'https://palmaryfood.com/wp-content/uploads/2021/04/Regalo-Digestif-600x600.jpg'],
  ['regalo digestive – chocolate',
   'https://palmaryfood.com/wp-content/uploads/2021/04/Regalo-chocolat-600x600.jpg'],
  ['regalo digestive – milk',
   'https://palmaryfood.com/wp-content/uploads/2021/04/Regalo-lait-600x600.jpg'],
  ['regalo digestive – peanut',
   'https://palmaryfood.com/wp-content/uploads/2021/04/Regalo-arachidt-600x600.jpg'],
  ['regalo hazelnut – 4x',
   'https://palmaryfood.com/wp-content/uploads/2021/04/Regalo-noisette-x4-600x600.jpg'],
  ['regalo hazelnut – 8x',
   'https://palmaryfood.com/wp-content/uploads/2021/04/Regalo-noisette-x8-600x600.jpg'],
  ['regalo hazelnut – family',
   'https://palmaryfood.com/wp-content/uploads/2021/04/Regalo-nois

In [24]:
product_table = [[k,p[0],p[1]] for k,v in product_dict.items() for p in v]

product_table = pd.DataFrame(product_table , columns=["category","name","img"])

product_table

,category,name,img
0,regalo,regalo butter,https://palmaryfood.com/wp-content/uploads/202...
1,regalo,regalo cookies,https://palmaryfood.com/wp-content/uploads/202...
2,regalo,regalo digestive,https://palmaryfood.com/wp-content/uploads/202...
3,regalo,regalo digestive – chocolate,https://palmaryfood.com/wp-content/uploads/202...
4,regalo,regalo digestive – milk,https://palmaryfood.com/wp-content/uploads/202...
...,...,...,...
97,moment,moment hazelnut chips,https://palmaryfood.com/wp-content/uploads/202...
98,moment,moment praline,https://palmaryfood.com/wp-content/uploads/202...
99,moment,moment white biscuits chips,https://palmaryfood.com/wp-content/uploads/202...
100,moment,moment white praline,https://palmaryfood.com/wp-content/uploads/202...


In [25]:
#creates a taste (Gout) column

for index, row in product_table.iterrows():

    category = row['category']
    name = row['name']
    
    # Remove the category from the name (if found)
    name = name.replace(category, '').strip()
    
    # Update the DataFrame with the modified name
    product_table.at[index, 'name'] = name

product_table["taste"] = product_table["name"]    

# pr1["imagelink"] = images_links



In [26]:

product_table= product_table[["category","name","taste","img"]]

product_table

,category,name,taste,img
0,regalo,butter,butter,https://palmaryfood.com/wp-content/uploads/202...
1,regalo,cookies,cookies,https://palmaryfood.com/wp-content/uploads/202...
2,regalo,digestive,digestive,https://palmaryfood.com/wp-content/uploads/202...
3,regalo,digestive – chocolate,digestive – chocolate,https://palmaryfood.com/wp-content/uploads/202...
4,regalo,digestive – milk,digestive – milk,https://palmaryfood.com/wp-content/uploads/202...
...,...,...,...,...
97,moment,hazelnut chips,hazelnut chips,https://palmaryfood.com/wp-content/uploads/202...
98,moment,praline,praline,https://palmaryfood.com/wp-content/uploads/202...
99,moment,white biscuits chips,white biscuits chips,https://palmaryfood.com/wp-content/uploads/202...
100,moment,white praline,white praline,https://palmaryfood.com/wp-content/uploads/202...


In [28]:
product_table.to_csv(r"products_new.csv",index=False)

In [75]:
produits = pd.read_csv(r"C:\Users\HP\Desktop\Brainthon\products_new.csv")
produits

,Unnamed: 0,category,name,Gout,image
0,0,regalo,regalo,butter,https://palmaryfood.com/wp-content/uploads/202...
1,1,regalo,cookies,basic,https://palmaryfood.com/wp-content/uploads/202...
2,2,regalo,digestive,digestive,https://palmaryfood.com/wp-content/uploads/202...
3,3,regalo,digestive,chocolate,https://palmaryfood.com/wp-content/uploads/202...
4,4,regalo,digestive,milk,https://palmaryfood.com/wp-content/uploads/202...
5,5,regalo,digestive,peanut,https://palmaryfood.com/wp-content/uploads/202...
6,6,regalo,regalo,hazelnut,https://palmaryfood.com/wp-content/uploads/202...
7,9,kool,spread,basic,https://palmaryfood.com/wp-content/uploads/202...
8,10,kool,break,chocolate,https://palmaryfood.com/wp-content/uploads/202...
9,12,kool,kool,chocolate,https://palmaryfood.com/wp-content/uploads/202...


In [78]:
products1=pr1["name"]
products1 = products1.drop([12,13,6,7,19,17,36,37,38])

products1

0                       butter
1                      cookies
2                    digestive
3        digestive – chocolate
4             digestive – milk
5           digestive – peanut
8            hazelnut – family
9                     – spread
10                       break
11    chocolate – large format
14    raspberry – large format
15                raspberry 4x
16                raspberry 8x
18             dondy – caramel
20           dondy – raspberry
21          dondy – strawberry
22               dream – choco
23           dream – raspberry
24          dream – strawberry
25             dream – vanilia
26                     kremali
27             kremali – choco
28            kremali – orange
29               bar – caramel
30                  bar – dark
31              bar – hazelnut
32                  bar – milk
33                bar – orange
34            bar – strawberry
35      chocolate spread – 1kg
39               coated cookie
40                coated wafer
41      

In [79]:
products1 = products1.reset_index(drop=True)


produits["real_name"]=products1


In [88]:
# produits.rename("Unnamed: 0")
produits.columns.values[0] = "id"
produits

,id,category,name,Gout,image,real_name
0,0,regalo,regalo,butter,https://palmaryfood.com/wp-content/uploads/202...,butter
1,1,regalo,cookies,basic,https://palmaryfood.com/wp-content/uploads/202...,cookies
2,2,regalo,digestive,digestive,https://palmaryfood.com/wp-content/uploads/202...,digestive
3,3,regalo,digestive,chocolate,https://palmaryfood.com/wp-content/uploads/202...,digestive – chocolate
4,4,regalo,digestive,milk,https://palmaryfood.com/wp-content/uploads/202...,digestive – milk
5,5,regalo,digestive,peanut,https://palmaryfood.com/wp-content/uploads/202...,digestive – peanut
6,6,regalo,regalo,hazelnut,https://palmaryfood.com/wp-content/uploads/202...,hazelnut – family
7,9,kool,spread,basic,https://palmaryfood.com/wp-content/uploads/202...,– spread
8,10,kool,break,chocolate,https://palmaryfood.com/wp-content/uploads/202...,break
9,12,kool,kool,chocolate,https://palmaryfood.com/wp-content/uploads/202...,chocolate – large format


In [91]:
produits.to_csv(r"C:\Users\HP\Desktop\Brainthon\products_new.csv",index=False)